<a href="https://colab.research.google.com/github/mlluizza/AIRBNB-RIO-DE-JANEIRO/blob/main/AIRBNB_RIO_DE_JANEIRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns**

# **Contexto**

No Airbnb, qualquer pessoa que tenha um quarto um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles(e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio,desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes exras, exigências de verificaçã de identidade do locador, etc.


# **Objetivo**

Construir um modelo de previsão de preço que permita um pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesma características) ou não.

**Importar Biblioteca e Bases de Dados**

In [ ]:
import pandas as pd
import pathlib

In [ ]:
meses = {'jan': 1, 'fev':2, 'mar':3, 'abr':4, 'mai':5,'jun':6, 'jul':7, 'ago':8,'set':9, 'out':10,'nov':11, 'dez':12}

from google.colab import drive
drive.mount('/content/gdrive')
caminho_bases = pathlib.Path('/content/gdrive/MyDrive/dataset')

base_airbnb = pd.DataFrame()
for arquivo in caminho_bases.iterdir():
  if arquivo.is_file() and arquivo.suffix == '.csv':
    nome_mes = arquivo.name[:3]
    if nome_mes in meses:
      mes = meses[nome_mes]

      df = pd.read_csv(caminho_bases / arquivo.name)
      base_airbnb = pd.concat([base_airbnb, df])

display(base_airbnb)

*   Temos muitas colunas,  o modelo pode acabar ficando muito lento.
*   Além disso uma análise rápida permite ver que várias colunas não são necessárias para o modelo de previsão, por isso irei excluir algumas colunas da nossa base.

*   Tipo de colunas que irei excluir:
1. IDs, Links e informações não relevantes para o modelo   
2. Colunas repetidas ou extremamente parecidas com outra (que dão a mesma informação para o modelo. Ex: Data x Ano/Mês)
3. Colunas preenchidas com texto livre ->  Não rodarei nenhuma análise de palavras ou algo do tipo
4. Colunas em que todos ou quase todos os valores são iguais.

*   Para isso, irei criar um arquivo em excel com 1.000 primeiros  registros e fazer uma análise qualitativa








In [ ]:
from os import sep
print(list(base_airbnb.columns))
base_airbnb.head(1000).to_csv('primeiros_registros.csv', sep=';')

In [ ]:
from google.colab import files
files.download('primeiros_registros.csv')

**Se tiver muitas colunas, irei indentificar quais colunas podemos excluir**

In [ ]:
print(base_airbnb['host_acceptance_rate'].value_counts())

In [ ]:
print((base_airbnb['host_listings_count']== base_airbnb['host_total_listings_count']).value_counts())

In [ ]:
print(base_airbnb['square_feet'].isnull().sum())